In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from laughing import phi15, utils

/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-09-27 06:15:39.562561: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-27 06:15:40.548410: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/intel/compilers_and_libraries_2018.3.222/linux/mpi/intel64/lib:/opt/intel/compilers_and_libraries_2018.3.222/linux/mpi/mic/lib:/o

In [22]:
model, _, tokenizer = phi15.load_model_and_tokenizer()

loading model...
loading tokenizer...


In [13]:
import json
# this file contains original instruction and test cases generated by fine-tuned phi1_5 and gpt4 teacher
test_cases_file = '../data/phi15_generated_test_cases_train.json'

with open(test_cases_file, encoding='utf-8') as fin:
  data = json.load(fin)

In [8]:
len(data)

3297

In [9]:
data[0]

{'instruction': 'Fix the following function to make it comply with PEP8 standards.',
 'input': 'def f(x):\nk= x+4\nreturn k',
 'output': 'def f(x):\n    k = x + 4\n    return k',
 'test_cases': 'def test_positive_number():\n    assert f(5) == 9\n\ndef test_negative_number():\n    assert f(-3) == 1\n\ndef test_zero():\n    assert f(0) == 4\n\ndef test_float_number():\n    assert f(2.5) == 6.5\n',
 'phi_finetuned_test_cases': "Generate unit tests for the below problem and its solution in Python.\n    Write each unit test as a seperate Python function with meaningful name that starts with 'test_'.\n\n    Problem:\n    Fix the following function to make it comply with PEP8 standards.\n\n    Solution:\n    def f(x):\n    k = x + 4\n    return k\n\n    Test cases:\n    def test_generate_sum_of_squares():\n        assert f(5) == 25\n        assert f(6) == 41\n        assert f(7) == 100\n        assert f(8) == 121\n        assert f(9) == 121\n        assert f(10) == 121\n        assert f(11) =

In [35]:
# generate outputs for the entire dataset
batch_size = 15
responses = []

def generate(data, responses=[]):
    for i in range(len(responses), len(data), batch_size):
        print(i, end=' ')
        prompts = phi15.make_prompts_for_code_gen(data[i:i+batch_size])
        responses.extend(phi15.gen_n_prompts(model, tokenizer, prompts))
    return responses


In [40]:
responses = generate(data, responses)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


840 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


855 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

1035 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


1050 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


1065 

In [ ]:
for x in responses:
    print(x, '\n========================\n')

In [ ]:
# save this data
# load phi15 fine-tuned model and generate test cases
# run batch eval for test cases
# prepare training for RRTH
# train model
